# Random Forest Classifier 모델
- 앙상블 모델이기 때문에 여러 개의 Decision Tree를 사용하여 학습하며, 이는 과적합을 방지
- Decision Tree와 마찬가지로, 비선형적인 분류 문제에 적합
- Feature importance를 측정할 수 있으므로, 변수 선택에 대한 정보를 제공
- 대체로 높은 정확도를 보이지만, 데이터에 따라 성능 차이가 크게 나타남
- 데이터가 많고 다양할수록 더욱 좋은 성능

In [1]:
#data analytics
import pandas as pd
from scipy.stats import pearsonr

#sklearn
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# # 토큰화
# import nltk
# nltk.download('punkt')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/데이터_전처리_파일.csv', encoding='cp949')

## 머신러닝 학습
- 상관관계가 높은 순서대로 변수 삽입
- 견적화폐(0.33), Machinery(0.17), Subject(0.13) 
- Assembly, 출고운반선이 추가 됐을 때 더 높은 정확도를 가진다.

In [16]:
# 텍스트 데이터
data = list( df['Subject'] + ' ' + df['출고운반선'] + ' ' + df['Control No.'] )

# target 값
target =list(df['리드타임_음수제거'])

# CountVectorizer를 이용하여 feature 벡터 생성
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)

# df[리드타임_음수제거]는 별도의 인코딩이 필요치않음
y = target

# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 정확도 함수
from sklearn.metrics import accuracy_score 
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score

def print_metrics(y_test, X_test, model) :
  # 테스트 데이터셋으로 예측
  y_pred = model.predict(X_test)

  # 각종 평가지표 계산
  recall = recall_score(y_test, y_pred, average='macro')
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='macro')
  f1 = f1_score(y_test, y_pred, average='macro')

  print("Recall: {:.4f}".format(recall))
  print("Accuracy: {:.4f}".format(accuracy))
  print("Precision: {:.4f}".format(precision))
  print("F1-score: {:.4f}".format(f1))

### Random Forest Classifier모델


In [15]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 모델 생성 및 학습
rfc = RandomForestClassifier(n_estimators=7, random_state=42)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)
rfc_acc = accuracy_score(y_test, rfc_pred)
print_metrics(y_test, X_test, rfc)

Recall: 0.8199
Accuracy: 0.9281
Precision: 0.8395
F1-score: 0.8258


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# 피클로 모델 저장
import pickle 
with open('lead_RFC.pickle','wb') as fw:
    pickle.dump(rfc, fw)

In [18]:
# vectorizer 생성
vectorizer = CountVectorizer()

# feature 벡터 생성
X = vectorizer.fit_transform(data)

# vectorizer 객체 저장
with open('lead_RFC_vectorizer.pickle', 'wb') as fw:
    pickle.dump(vectorizer, fw)